In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import syft as sy
import sklearn
import torch
from sklearn.datasets import load_boston
from matplotlib import pyplot as plt
from torch.utils.data import Dataset

In [ ]:
duet = sy.launch_duet(loopback=True)

## Checkpoint#1

## Run DS.ipynb until Checkpoint#1

Data owner create Dataset object and save it in a .pt file.

In [ ]:
dataset = load_boston()
X = torch.tensor(dataset["data"]).float()
y = torch.tensor(dataset["target"]).float()

class BostonDS(Dataset):
    def __init__(self, X,y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, i):
        return (self.X[i], self.y[i])
    
ds = BostonDS(X,y)
torch.save(ds, "boston_ds.pt")

Data owner send the path string of this .pt file to data scientist.

In [ ]:
sy.lib.python.String("boston_ds.pt").send(duet, pointable=True, tags=["meta"])

Data owner add the following request handler to allow data scientist to iterate on the RemoteDataLoader object.

Why `tags=["meta","RemoteDataset", "RemoteDataLoader", "__len__"]`? Because a RemoteDataset object `rds` will be created from the meta info, which has `tags=["meta"]`, so `rds` will inherite this tag, plus the constructor method name `RemoteDataset`. Then, the RemoteDataLoader object `rdl` will inherite these tags, plus the constructor method name `RemoteDataLoader`. Then, `rdl.__len__()` will inherite these tags, plus the method name `__len__`.

In [ ]:
duet.requests.add_handler(
    action="accept",
    tags=[
        "meta", "RemoteDataset", "RemoteDataLoader", "__len__"
    ])

Data owner add the following request handler to allow data scientist to `get` the `loss`.

The `tags` used here comes from all the names of methods being called to get a `loss`.

In [ ]:
duet.requests.add_handler(
    action="accept",
    tags=[
        'meta',
        'RemoteDataset',
        'RemoteDataLoader',
        '__iter__',
        '__next__',
        '__getitem__',
        '__call__',
        'view',
        '__sub__',
        '__pow__',
        'mean'])

## Run DS.ipynb until the end

In [ ]:
duet.store.pandas